In [2]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt

In [3]:
n, l, m = (10 - 1) * 2 + 2, 5 + (10 - 1) + (10)*2, 10 # количество видов узлов и возможных взаимодействий между ними
alpha = np.array([10, 10])
lichers = np.array([i for i in range(m - 1)])
lichers_ = np.append(
    (
            np.cumsum(
                np.append(lichers, alpha[1]
                          )[::-1]
            )[::-1]
    )[:-1]
    , alpha[1]
) # новые клиенты, сиды, личеры имеющие i=1,..,m-1 частей файла, личеры, интересующиеся i=1,...,m-1,m частями файла
# пример личеров: lichers = [0 1 2 3 4 5 6 7 8]; lichers_ = [46 46 45 43 40 36 31 25 18 10];
# lichers_[-1] = alpha[1] - сиды, lichers_[0] = sum(lichers) + alpha[1] - сумма личеров, интересующих новых клиентов.
# lichers_[1] - сумма личеров, интересующих первого личера (lichers[0]) и т.д.
lmb = 2
beta = 0.2
delta = sum(lichers_) * 4
mu = 2
phi = ([lmb, beta * lichers_[0] * alpha[0]] +
       [lichers_[i] * lichers[i - 1] / delta for i in range(1, m)] +
       [mu])

eps = np.array([[0, 0] + [0 for i in range(m - 1)] + [0 for i in range(m)]] +
                [[1, 0] + [0 for i in range(m - 1)] + [1 if i == 0 else 0 for i in range(m)]] +
                [[0, 0] + [1 if i == j else 0 for i in range(m - 1)] + [1 if i == j else 0 for i in range(m)] for j in range(m - 1)] +
                [[0, 1] + [0 for i in range(m - 1)] + [0 for i in range(m - 1)]])

p_gamma = np.array([[1, 0]+ [0 for i in range(m - 1)] + [0 for i in range(m)]] +
                   [[]] +
                   [[]] +
                   [[]])

In [52]:
n, l, m = m + 1, m + 2, 10 # количество видов узлов и возможных взаимодействий между ними

lichers = np.array([m if i == 0 or i == m else i for i in range(m + 1)])
lichers_ = np.append(np.cumsum(lichers[::-1]
            )[:-1:][::-1], 0)
# новые клиенты, сиды, личеры имеющие i=1,..,m-1 частей файла, личеры, интересующиеся i=1,...,m-1,m частями файла
# пример личеров: lichers = [0 1 2 3 4 5 6 7 8]; lichers_ = [46 46 45 43 40 36 31 25 18 10];
# lichers_[-1] = alpha[1] - сиды, lichers_[0] = sum(lichers) + alpha[1] - сумма личеров, интересующих новых клиентов.
# lichers_[1] - сумма личеров, интересующих первого личера (lichers[0]) и т.д.
lmb = 2
delta = sum(lichers_)
mu = 2
phi = ([lmb] +
       [lichers_[i] * lichers[i] / delta for i in range(m)] +
       [mu])

eps = np.array([[0 for i in range(m + 1)] + [0 for i in range(m + 1)]] +
               [[1 if i == j else 0 for i in range(m + 1)] + [1 if i == j else 0 for i in range(m + 1)] for j in range(m)] +
               [[1 if i == m else 0 for i in range(m + 1)] + [0 for i in range(m + 1)]])

p_gamma = np.array([[1 if i == 0 else 0 for i in range(m + 1)] + [0 for i in range(m + 1)]] +
                   [[1 if i == j + 1 else 0 for i in range(m + 1)] + [1 if i <= j else 0 for i in range(m + 1)] for j in range(m)] +
                   [[0 for i in range(m + 1)] + [0 for i in range(m + 1)]])

In [54]:
def inverse_exp(random_, coef):
    '''возвращает значение случайной величины,
    распределённой по экспоненциальному закону,
    соответствующему простейшему Пуассоновскому потоку'''
    if coef == 0:
        return np.finfo(np.float32).max
    return -1/coef * math.log(1 - random_)

tau = np.array([0 for _ in range(l)], dtype=np.float32) # время следующего события i
for i in range(l):
    tau[i] = inverse_exp(random.random(), phi[i])

def update_tau(tau, ):
    ''' Возвращает матрицу с временами событий на следующем шаге и ближайшее событие
    :param tau: array of trigger times, generated on the previous step
    :return: times array on the next step, next event, time of next event'''
    min_ = np.finfo(np.float32).max
    idx_min = 0

    for k in range(len(tau)):
        if tau[k] == np.finfo(np.float32).max and phi[k] != 0:
            tau[k] = inverse_exp(random.random(), phi[k])
        if tau[k] < min_:
            min_ = tau[k]
            idx_min = k
    tau -= min_
    return tau, idx_min, min_

In [55]:
print(tau)
print(eps)
print(p_gamma)

[1.0338802e+00 2.1941154e-03 2.6015096e+00 9.0951729e+00 3.0597312e+00
 3.1093365e-01 2.1724936e-01 7.4188375e-01 7.6993096e-01 1.4202839e-01
 1.7948222e+00 9.4992393e-01]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 1 